In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import minmax_scale

# Import Data
Found on [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset), collected from [TMDB](https://www.themoviedb.org/?language=en) and [GroupLens](https://grouplens.org/datasets/movielens/latest/).

In [2]:
#Start importing some of the features available
df = pd.read_csv('data/movies_metadata.csv',dtype={'original_language': 'str','poster_path': 'str'},low_memory=False)
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [3]:
#Have a look at some of the data
#df.head()

In [4]:
df.shape

(45466, 24)

Approx 45k movies with 24 features 

### Some cleanup

Largely taken from [this notebook](https://www.kaggle.com/rounakbanik/the-story-of-film) accompanying dataset on [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset).

In [5]:
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['revenue'] = df['revenue'].replace(0, np.nan)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].replace(0, np.nan)
df = df.drop(['original_title'], axis=1)
df = df.drop(['poster_path'], axis=1)
df = df.drop('adult', axis=1)

Add new feature -- percent profit

In [6]:
df['return'] = df['revenue'] / df['budget']
df['budget'] = df['budget'].replace(0, np.nan)
df[df['return'].isnull()].shape

(40085, 22)

Parse year

In [7]:
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
df['year'] = pd.to_numeric(df['year'], errors='coerce')

Clean up the json stuff

In [8]:
df['genres'] = df['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

Features related to reception of film online

In [9]:
def clean_numeric(x):
    try:
        return float(x)
    except:
        return np.nan

In [10]:
df['popularity'] = df['popularity'].apply(clean_numeric).astype('float')
df['vote_count'] = df['vote_count'].apply(clean_numeric).astype('float')
df['vote_average'] = df['vote_average'].apply(clean_numeric).astype('float')

In [11]:
df.shape

(45466, 23)

### Add credits data

In [12]:
credits = pd.read_csv('data/credits.csv')

In [13]:
credits['id'] = credits['id'].astype('int')

In [14]:
#Remove some buggy data, luckily not needed for this tournament
df=df[df['id']!='1997-08-20']#bug in data?
df=df[df['id']!='2012-09-29']#bug in data?
df=df[df['id']!='2014-01-01']#bug in data?

In [15]:
credits['id'] = credits['id'].astype('int')
df['id'] = df['id'].astype('int')

In [16]:
df = df.merge(credits, on='id')

In [17]:
df.shape

(45538, 25)

In [18]:
df['cast_size'] = df['cast'].apply(lambda x: len(x))
df['crew_size'] = df['crew'].apply(lambda x: len(x))

### Drop all but the features we'll want later and some final cleaning

In [19]:
md=df[['id','title','year','return','budget','runtime','cast_size','crew_size','popularity','vote_count', 'vote_average']]

#Remove NaN
md = md.fillna(-1) #replace NaN with -1

#Normalize the numbers, except year.  do that one later.
md[['return','budget','runtime','cast_size','crew_size','popularity','vote_count', 'vote_average']] = minmax_scale(md[['return','budget','runtime','cast_size','crew_size','popularity','vote_count', 'vote_average']])


In [20]:
md.shape


(45538, 11)

## Construct Filmspotting Madness Bracket

In [21]:
def make_match(md, title1, title2, title1win):
    df1=md[(md['title']==title1) & (md['year'] > 1989) & (md['year'] < 2000)]
    df1 = df1.add_suffix('_1')
    df1 = df1.reset_index(drop=True)

    df2=md[(md['title']==title2) & (md['year'] > 1989) & (md['year'] < 2000)]
    df2 = df2.add_suffix('_2')
    df2 = df2.reset_index(drop=True)

    my_match=df1.join(df2)

    my_match['title_1_wins']=title1win
        
    return my_match

In [22]:
#Round 1
my_match = make_match(md, "Pulp Fiction", "Wild at Heart", 1)
my_match = my_match.append( make_match(md, "Saving Private Ryan", "Out of Sight", 1) )
my_match = my_match.append( make_match(md, "Toy Story", "The Sixth Sense", 1) )
my_match = my_match.append( make_match(md, "Schindler's List", "The Iron Giant", 1) )
my_match = my_match.append( make_match(md, "Fight Club", "Safe", 1) )
my_match = my_match.append( make_match(md, "Barton Fink", "Leon: The Professional", 1) )
my_match = my_match.append( make_match(md, "Chungking Express", "My Own Private Idaho", 1) )
my_match = my_match.append( make_match(md, "Jackie Brown", "Princess Mononoke", 1) )
my_match = my_match.append( make_match(md, "The Silence of the Lambs","Close-Up", 1) )
my_match = my_match.append( make_match(md, "Terminator 2: Judgment Day", "The Lion King", 1) )
my_match = my_match.append( make_match(md, "Heat", "JFK", 1) )
my_match = my_match.append( make_match(md, "Dazed and Confused", "The Player", 1) )
my_match = my_match.append( make_match(md, "Boogie Nights", "Naked", 1) )
my_match = my_match.append( make_match(md, "Malcolm X", "Blue", 1) )
my_match = my_match.append( make_match(md, "The Big Lebowski", "The Truman Show", 1) )
my_match = my_match.append( make_match(md, "Groundhog Day", "Edward Scissorhands", 1) )
my_match = my_match.append( make_match(md, "GoodFellas", "There's Something About Mary", 1) )
my_match = my_match.append( make_match(md, "Jurassic Park", "Miller's Crossing", 1) )
my_match = my_match.append( make_match(md, "Magnolia", "All About My Mother", 1) )
my_match = my_match.append( make_match(md, "Being John Malkovich", "Office Space", 1) )
my_match = my_match.append( make_match(md, "Unforgiven", "Metropolitan", 1) )
my_match = my_match.append( make_match(md, "Trainspotting", "Clueless", 1) )
my_match = my_match.append( make_match(md, "The Matrix", "The Blair Witch Project", 1) )
my_match = my_match.append( make_match(md, "Hoop Dreams", "The Thin Red Line", 1) )
my_match = my_match.append( make_match(md, "Fargo", "Dead Man", 1) )
my_match = my_match.append( make_match(md, "L.A. Confidential", "Glengarry Glen Ross", 1) )
my_match = my_match.append( make_match(md, "Before Sunrise", "The Insider", 1) )
my_match = my_match.append( make_match(md, "Eyes Wide Shut", "The Piano", 1) )
my_match = my_match.append( make_match(md, "Rushmore", "The Sweet Hereafter", 1) )
my_match = my_match.append( make_match(md, "The Shawshank Redemption", "The Usual Suspects", 1) )
my_match = my_match.append( make_match(md, "Reservoir Dogs", "Boyz n the Hood", 1) )
my_match = my_match.append( make_match(md, "Se7en", "Starship Troopers", 1) )

#Round 2
my_match = my_match.append( make_match(md, "Pulp Fiction", "Saving Private Ryan", 1) )
my_match = my_match.append( make_match(md, "Toy Story", "Schindler's List", 1) )
my_match = my_match.append( make_match(md, "Fight Club", "Barton Fink", 1) )
my_match = my_match.append( make_match(md, "Jackie Brown", "Chungking Express", 1) )
my_match = my_match.append( make_match(md, "The Silence of the Lambs", "Terminator 2: Judgment Day", 1) )
my_match = my_match.append( make_match(md, "Dazed and Confused", "Heat", 1) )
my_match = my_match.append( make_match(md, "Boogie Nights", "Malcolm X", 1) )
my_match = my_match.append( make_match(md, "The Big Lebowski", "Groundhog Day", 1) )
my_match = my_match.append( make_match(md, "GoodFellas", "Jurassic Park", 1) )
my_match = my_match.append( make_match(md, "Magnolia", "Being John Malkovich", 1) )
my_match = my_match.append( make_match(md, "Unforgiven", "Trainspotting", 1) )
my_match = my_match.append( make_match(md, "The Matrix", "Hoop Dreams", 1) )
my_match = my_match.append( make_match(md, "Fargo", "L.A. Confidential", 1) )
my_match = my_match.append( make_match(md, "Before Sunrise", "Eyes Wide Shut", 1) )
my_match = my_match.append( make_match(md, "Rushmore", "The Shawshank Redemption", 1) )
my_match = my_match.append( make_match(md, "Reservoir Dogs", "Se7en", 1) )

#Round 3
my_match = my_match.append( make_match(md, "Pulp Fiction", "Toy Story", 1) )
my_match = my_match.append( make_match(md, "Fight Club", "Jackie Brown", 1) )
my_match = my_match.append( make_match(md, "The Silence of the Lambs", "Dazed and Confused", 1) )
my_match = my_match.append( make_match(md, "The Big Lebowski", "Boogie Nights", 1) )
my_match = my_match.append( make_match(md, "GoodFellas", "Being John Malkovich", 1) )
my_match = my_match.append( make_match(md, "The Matrix", "Unforgiven", 1) )
my_match = my_match.append( make_match(md, "Fargo", "Before Sunrise", 1) )
my_match = my_match.append( make_match(md, "Rushmore", "Se7en", 1) )

#Round 4
my_match = my_match.append( make_match(md, "Pulp Fiction", "Fight Club", 1) )
my_match = my_match.append( make_match(md, "The Silence of the Lambs", "The Big Lebowski", 1) )
my_match = my_match.append( make_match(md, "GoodFellas", "The Matrix", 1) )
my_match = my_match.append( make_match(md, "Fargo", "Rushmore", 1) )

#Round 5
my_match = my_match.append( make_match(md, "Pulp Fiction", "The Silence of the Lambs", 1) )
my_match = my_match.append( make_match(md, "Fargo", "GoodFellas", 1) )


#Have a look
#my_match

#### Add same matches with movies in reverse order so NN doesn't learn preference for arbitrary movie number ordering
NB: we remove duplicates between training and test data

In [23]:
#Round 1
my_match = my_match.append( make_match(md, "Wild at Heart", "Pulp Fiction", 0) )
my_match = my_match.append( make_match(md, "Out of Sight", "Saving Private Ryan", 0) )
my_match = my_match.append( make_match(md, "The Sixth Sense", "Toy Story", 0) )
my_match = my_match.append( make_match(md, "The Iron Giant", "Schindler's List", 0) )
my_match = my_match.append( make_match(md, "Safe", "Fight Club", 0) )
my_match = my_match.append( make_match(md, "Leon: The Professional", "Barton Fink", 0) )
my_match = my_match.append( make_match(md, "My Own Private Idaho", "Chungking Express", 0) )
my_match = my_match.append( make_match(md, "Princess Mononoke", "Jackie Brown", 0) )
my_match = my_match.append( make_match(md, "Close-Up", "The Silence of the Lambs", 0) )
my_match = my_match.append( make_match(md, "The Lion King", "Terminator 2: Judgment Day", 0) )
my_match = my_match.append( make_match(md, "JFK", "Heat", 0) )
my_match = my_match.append( make_match(md, "The Player", "Dazed and Confused", 0) )
my_match = my_match.append( make_match(md, "Naked", "Boogie Nights", 0) )
my_match = my_match.append( make_match(md, "Blue", "Malcolm X", 0) )
my_match = my_match.append( make_match(md, "The Truman Show", "The Big Lebowski", 0) )
my_match = my_match.append( make_match(md, "Edward Scissorhands", "Groundhog Day", 0) )
my_match = my_match.append( make_match(md, "There's Something About Mary", "GoodFellas", 0) )
my_match = my_match.append( make_match(md, "Miller's Crossing", "Jurassic Park", 0) )
my_match = my_match.append( make_match(md, "All About My Mother", "Magnolia", 0) )
my_match = my_match.append( make_match(md, "Office Space", "Being John Malkovich", 0) )
my_match = my_match.append( make_match(md, "Metropolitan", "Unforgiven", 0) )
my_match = my_match.append( make_match(md, "Clueless", "Trainspotting", 0) )
my_match = my_match.append( make_match(md, "The Blair Witch Project", "The Matrix", 0) )
my_match = my_match.append( make_match(md, "The Thin Red Line", "Hoop Dreams", 0) )
my_match = my_match.append( make_match(md, "Dead Man", "Fargo", 0) )
my_match = my_match.append( make_match(md, "Glengarry Glen Ross", "L.A. Confidential", 0) )
my_match = my_match.append( make_match(md, "The Insider", "Before Sunrise", 0) )
my_match = my_match.append( make_match(md, "The Piano", "Eyes Wide Shut", 0) )
my_match = my_match.append( make_match(md, "The Sweet Hereafter", "Rushmore", 0) )
my_match = my_match.append( make_match(md, "The Usual Suspects", "The Shawshank Redemption", 0) )
my_match = my_match.append( make_match(md, "Boyz n the Hood", "Reservoir Dogs", 0) )
my_match = my_match.append( make_match(md, "Starship Troopers", "Se7en", 0) )

#Round 2
my_match = my_match.append( make_match(md, "Saving Private Ryan", "Pulp Fiction", 0) )
my_match = my_match.append( make_match(md, "Schindler's List", "Toy Story", 0) )
my_match = my_match.append( make_match(md, "Barton Fink", "Fight Club", 0) )
my_match = my_match.append( make_match(md, "Chungking Express", "Jackie Brown", 0) )
my_match = my_match.append( make_match(md, "Terminator 2: Judgment Day", "The Silence of the Lambs", 0) )
my_match = my_match.append( make_match(md, "Heat", "Dazed and Confused", 0) )
my_match = my_match.append( make_match(md, "Malcolm X", "Boogie Nights", 0) )
my_match = my_match.append( make_match(md, "Groundhog Day", "The Big Lebowski", 0) )
my_match = my_match.append( make_match(md, "Jurassic Park", "GoodFellas", 0) )
my_match = my_match.append( make_match(md, "Being John Malkovich", "Magnolia", 0) )
my_match = my_match.append( make_match(md, "Trainspotting", "Unforgiven", 0) )
my_match = my_match.append( make_match(md, "Hoop Dreams", "The Matrix", 0) )
my_match = my_match.append( make_match(md, "L.A. Confidential", "Fargo", 0) )
my_match = my_match.append( make_match(md, "Eyes Wide Shut", "Before Sunrise", 0) )
my_match = my_match.append( make_match(md, "The Shawshank Redemption", "Rushmore", 0) )
my_match = my_match.append( make_match(md, "Se7en", "Reservoir Dogs", 0) )

#Round 3
my_match = my_match.append( make_match(md, "Toy Story", "Pulp Fiction", 0) )
my_match = my_match.append( make_match(md, "Jackie Brown", "Fight Club", 0) )
my_match = my_match.append( make_match(md, "Dazed and Confused", "The Silence of the Lambs", 0) )
my_match = my_match.append( make_match(md, "Boogie Nights", "The Big Lebowski", 0) )
my_match = my_match.append( make_match(md, "Being John Malkovich", "GoodFellas", 0) )
my_match = my_match.append( make_match(md, "Unforgiven", "The Matrix", 0) )
my_match = my_match.append( make_match(md, "Before Sunri se", "Fargo", 0) )
my_match = my_match.append( make_match(md, "Se7en", "Rushmore", 0) )

#Round 4
my_match = my_match.append( make_match(md, "Fight Club", "Pulp Fiction", 0) )
my_match = my_match.append( make_match(md, "The Big Lebowski", "The Silence of the Lambs", 0) )
my_match = my_match.append( make_match(md, "The Matrix", "GoodFellas", 0) )
my_match = my_match.append( make_match(md, "Rushmore", "Fargo", 0) )

#Round 5
my_match = my_match.append( make_match(md, "The Silence of the Lambs", "Pulp Fiction", 0) )
my_match = my_match.append( make_match(md, "GoodFellas", "Fargo", 0) )

### Final match we want to predict

In [24]:
final_match = make_match(md, "Pulp Fiction", "Fargo", -1)

## Final feature selection and prep

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
#normalize the years, which we kept unormalized earlier to select on it

#Normalize year in final match
final_match[['year_1']] = (final_match[['year_1']] - my_match[['year_1']].min(axis=0)) / (my_match[['year_1']].max(axis=0) - my_match[['year_1']].min(axis=0))
final_match[['year_2']] = (final_match[['year_2']] - my_match[['year_2']].min(axis=0)) / (my_match[['year_2']].max(axis=0) - my_match[['year_2']].min(axis=0))

#Normalize year in training/test data
my_match[['year_1','year_2']] = minmax_scale(my_match[['year_1','year_2']])
final_match[['year_1','year_2']] = minmax_scale(final_match[['year_1','year_2']])

In [27]:
my_match

,id_1,title_1,year_1,return_1,budget_1,runtime_1,cast_size_1,crew_size_1,popularity_1,vote_count_1,...,year_2,return_2,budget_2,runtime_2,cast_size_2,crew_size_2,popularity_2,vote_count_2,vote_average_2,title_1_wins
0,680,Pulp Fiction,0.444444,2.237838e-06,0.021053,0.123309,0.180053,0.205560,0.258803,0.616013,...,0.000000,2.043062e-07,0.025000,0.099443,0.067157,0.034810,0.017373,0.024297,0.727273,1
0,857,Saving Private Ryan,0.888889,6.359469e-07,0.184211,0.135243,0.127186,0.262572,0.041492,0.365800,...,0.888889,2.113277e-07,0.126316,0.098648,0.053266,0.030177,0.015223,0.024084,0.681818,1
0,862,Toy Story,0.555556,1.085139e-06,0.078947,0.065235,0.043806,0.253427,0.041837,0.384768,...,1.000000,1.437529e-06,0.105263,0.085919,0.165878,0.208318,0.035460,0.229042,0.790909,1
0,424,Schindler's List,0.333333,1.259039e-06,0.057895,0.155927,0.101519,0.085191,0.077896,0.315217,...,1.000000,1.073577e-07,0.184211,0.069212,0.101919,0.107651,0.029976,0.104504,0.781818,1
0,550,Fight Club,1.000000,2.098074e-07,0.165789,0.111376,0.259469,0.250275,0.118270,0.687624,...,0.555556,0.000000e+00,0.000000,0.095465,0.041690,0.021250,0.016737,0.004476,0.745455,1
0,290,Barton Fink,0.111111,1.358276e-07,0.023684,0.093079,0.045821,0.038873,0.017114,0.032040,...,0.444444,3.089861e-07,0.042105,0.088305,0.129018,0.045001,0.039157,0.305058,0.836364,1
0,11104,Chungking Express,0.444444,0.000000e+00,0.000000,0.081941,0.038825,0.032337,0.014183,0.017761,...,0.111111,0.000000e+00,0.006579,0.083532,0.039492,0.025938,0.015114,0.015558,0.736364,1
0,184,Jackie Brown,0.777778,3.473672e-07,0.031579,0.123309,0.107632,0.022269,0.031752,0.112319,...,0.777778,5.658233e-07,0.069737,0.107399,0.047603,0.160586,0.033121,0.145070,0.836364,1
0,274,The Silence of the Lambs,0.111111,1.238659e-06,0.050000,0.095465,0.191995,0.193345,0.009676,0.323245,...,0.000000,0.000000e+00,0.000000,0.078759,0.047553,0.015027,0.007242,0.003836,0.818182,1
0,280,Terminator 2: Judgment Day,0.111111,5.001458e-07,0.263158,0.109785,0.136663,0.127542,0.043140,0.303708,...,0.444444,1.493700e-06,0.118421,0.071599,0.099837,0.084920,0.041215,0.392228,0.818182,1


In [28]:
#Simplest splitting -- random.  Could rethink this to e.g. keep final match movies equal in # of samples?
my_match_train, my_match_test = train_test_split(my_match, test_size=0.33, random_state=42)

In [29]:
my_match_train.shape

(82, 23)

In [30]:
my_match_test.shape

(41, 23)

In [31]:
#These are the train samples we will remove because they are in test set with reverse order
for index, row in my_match_train.iterrows(): 
    #print(row['title_1'], row['title_2'])
    for index2, row2 in my_match_test.iterrows():
        if(row['title_1']==row2['title_2'] and row['title_2']==row2['title_1']):
            print(row['title_1'], row['title_2'])

Rushmore The Sweet Hereafter
GoodFellas There's Something About Mary
All About My Mother Magnolia
Eyes Wide Shut The Piano
Terminator 2: Judgment Day The Silence of the Lambs
Eyes Wide Shut Before Sunrise
Schindler's List The Iron Giant
Boogie Nights Malcolm X
JFK Heat
Rushmore The Shawshank Redemption
The Insider Before Sunrise
Trainspotting Unforgiven
The Silence of the Lambs The Big Lebowski
Pulp Fiction Saving Private Ryan
Unforgiven The Matrix
Fargo Rushmore
The Shawshank Redemption The Usual Suspects
Se7en Reservoir Dogs
Naked Boogie Nights
Dead Man Fargo
The Lion King Terminator 2: Judgment Day
L.A. Confidential Fargo
The Big Lebowski The Truman Show
Boyz n the Hood Reservoir Dogs
Jurassic Park GoodFellas


In [32]:
#Here we remove from training set any of the reverse order duplicate matches that exist in the test set
keys1 = ['title_1', 'title_2']
keys2 = ['title_2', 'title_1']

i_train = my_match_train.set_index(keys1).index
i_test = my_match_test.set_index(keys2).index
my_match_train = my_match_train[~i_train.isin(i_test)]

In [33]:
#i_train

In [34]:
#i_test

In [35]:
#double check they're gone
for index, row in my_match_train.iterrows(): 
    #print(row['title_1'], row['title_2'])
    for index2, row2 in my_match_test.iterrows():
        if(row['title_1']==row2['title_2'] and row['title_2']==row2['title_1']):
            print(row['title_1'], row['title_2'])

### Set the features that will be trained on

In [36]:
#columns = ['year_1','budget_1','return_1','cast_size_1','crew_size_1','popularity_1','vote_count_1',"vote_average_1",'year_2','budget_2','return_2','cast_size_2','crew_size_2','popularity_2','vote_count_2',"vote_average_2"]
#columns = ['year_1','budget_1','return_1','cast_size_1','popularity_1','vote_count_1',"vote_average_1",'year_2','budget_2','return_2','cast_size_2','popularity_2','vote_count_2',"vote_average_2"]
#columns = ['return_1','popularity_1','vote_count_1','vote_average_1', 'cast_size_1','return_2','popularity_2','vote_count_2','vote_average_2', 'cast_size_2']
#columns = ['popularity_1','vote_count_1',"vote_average_1",'popularity_2','vote_count_2',"vote_average_2"]


#The best of the ones I tried:
columns = ['return_1','popularity_1','vote_count_1',"vote_average_1",'return_2','popularity_2','vote_count_2',"vote_average_2"]

In [37]:
X_train = my_match_train[columns]
X_test = my_match_test[columns]
X_final = final_match[columns]

In [38]:
X_train.shape

(57, 8)

In [39]:
X_test.shape

(41, 8)

In [40]:
y_train = my_match_train[['title_1_wins']]
y_test = my_match_test[['title_1_wins']]

In [41]:
X_train.shape[1:2]

(8,)

In [42]:
y_train.shape[1]

1

In [43]:
X_final

,return_1,popularity_1,vote_count_1,vote_average_1,return_2,popularity_2,vote_count_2,vote_average_2
0,0.000002,0.258803,0.616013,0.845455,7.791670e-07,0.024822,0.14784,0.790909


# Neural Network
Not optimal with so few features though

In [44]:

from sklearn import metrics
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential
import matplotlib
#from callbacks import all_callbacks #this doesn't work

/Users/kreis/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Setup Neural Network Model

In [45]:
model = Sequential()

In [46]:
#Not really ideal with so few samples...  Let's see how this goes
model.add(Dense(12, input_shape = X_train.shape[1:2], activation = 'relu', kernel_initializer='lecun_uniform'))
#model.add(Dense(12, activation = 'relu', kernel_initializer='lecun_uniform'))
#model.add(Dense(6, activation = 'relu', kernel_initializer='lecun_uniform'))
model.add(Dense(y_train.shape[1], activation = 'sigmoid', kernel_initializer='lecun_uniform'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [47]:
# For a binary classification problem (https://keras.io/getting-started/sequential-model-guide/)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [48]:
hist = model.fit(X_train, y_train, epochs=300, batch_size=10)


Epoch 1/300
57/57 [==============================] - 0s 2ms/step - loss: 0.7054 - acc: 0.5088
Epoch 2/300
57/57 [==============================] - 0s 197us/step - loss: 0.7023 - acc: 0.5088
Epoch 3/300
57/57 [==============================] - 0s 213us/step - loss: 0.7013 - acc: 0.5088
Epoch 4/300
57/57 [==============================] - 0s 190us/step - loss: 0.7004 - acc: 0.5088
Epoch 5/300
57/57 [==============================] - 0s 214us/step - loss: 0.6994 - acc: 0.5088
Epoch 6/300
57/57 [==============================] - 0s 224us/step - loss: 0.6987 - acc: 0.5088
Epoch 7/300
57/57 [==============================] - 0s 279us/step - loss: 0.6970 - acc: 0.5088
Epoch 8/300
57/57 [==============================] - 0s 513us/step - loss: 0.6959 - acc: 0.5088
Epoch 9/300
57/57 [==============================] - 0s 215us/step - loss: 0.6954 - acc: 0.5088
Epoch 10/300
57/57 [==============================] - 0s 313us/step - loss: 0.6953 - acc: 0.5088
Epoch 11/300
57/57 [=====================

57/57 [==============================] - 0s 281us/step - loss: 0.6388 - acc: 0.7368
Epoch 86/300
57/57 [==============================] - 0s 241us/step - loss: 0.6382 - acc: 0.7018
Epoch 87/300
57/57 [==============================] - 0s 231us/step - loss: 0.6374 - acc: 0.7018
Epoch 88/300
57/57 [==============================] - 0s 223us/step - loss: 0.6366 - acc: 0.7018
Epoch 89/300
57/57 [==============================] - 0s 253us/step - loss: 0.6358 - acc: 0.7193
Epoch 90/300
57/57 [==============================] - 0s 347us/step - loss: 0.6351 - acc: 0.7018
Epoch 91/300
57/57 [==============================] - 0s 310us/step - loss: 0.6345 - acc: 0.7018
Epoch 92/300
57/57 [==============================] - 0s 251us/step - loss: 0.6333 - acc: 0.7193
Epoch 93/300
57/57 [==============================] - 0s 201us/step - loss: 0.6330 - acc: 0.6842
Epoch 94/300
57/57 [==============================] - 0s 250us/step - loss: 0.6329 - acc: 0.7018
Epoch 95/300
57/57 [=======================

57/57 [==============================] - 0s 259us/step - loss: 0.5887 - acc: 0.7018
Epoch 169/300
57/57 [==============================] - 0s 305us/step - loss: 0.5876 - acc: 0.7544
Epoch 170/300
57/57 [==============================] - 0s 238us/step - loss: 0.5885 - acc: 0.7018
Epoch 171/300
57/57 [==============================] - 0s 227us/step - loss: 0.5877 - acc: 0.7193
Epoch 172/300
57/57 [==============================] - 0s 282us/step - loss: 0.5867 - acc: 0.7544
Epoch 173/300
57/57 [==============================] - 0s 284us/step - loss: 0.5860 - acc: 0.7544
Epoch 174/300
57/57 [==============================] - 0s 247us/step - loss: 0.5859 - acc: 0.7368
Epoch 175/300
57/57 [==============================] - 0s 191us/step - loss: 0.5849 - acc: 0.7368
Epoch 176/300
57/57 [==============================] - 0s 188us/step - loss: 0.5843 - acc: 0.7193
Epoch 177/300
57/57 [==============================] - 0s 222us/step - loss: 0.5846 - acc: 0.7193
Epoch 178/300
57/57 [=============

57/57 [==============================] - 0s 273us/step - loss: 0.5627 - acc: 0.7193
Epoch 252/300
57/57 [==============================] - 0s 219us/step - loss: 0.5634 - acc: 0.7193
Epoch 253/300
57/57 [==============================] - 0s 295us/step - loss: 0.5642 - acc: 0.7368
Epoch 254/300
57/57 [==============================] - 0s 291us/step - loss: 0.5625 - acc: 0.7368
Epoch 255/300
57/57 [==============================] - 0s 236us/step - loss: 0.5623 - acc: 0.7368
Epoch 256/300
57/57 [==============================] - 0s 225us/step - loss: 0.5634 - acc: 0.7368
Epoch 257/300
57/57 [==============================] - 0s 215us/step - loss: 0.5621 - acc: 0.7193
Epoch 258/300
57/57 [==============================] - 0s 267us/step - loss: 0.5624 - acc: 0.7368
Epoch 259/300
57/57 [==============================] - 0s 460us/step - loss: 0.5628 - acc: 0.7368
Epoch 260/300
57/57 [==============================] - 0s 369us/step - loss: 0.5614 - acc: 0.7544
Epoch 261/300
57/57 [=============

In [49]:
 model.evaluate(X_test, y_test, batch_size=10) #returns loss and accuracy

41/41 [==============================] - 0s 770us/step


[0.6115584606077613, 0.7317073170731707]

In [50]:
#confusion matrix
print(metrics.confusion_matrix(y_test, model.predict(X_test)>0.5)) 

[[15  4]
 [ 7 15]]


In [51]:
#y_test

In [52]:
#model.predict(X_test)

In [53]:
print(hist.history.keys())

dict_keys(['loss', 'acc'])


In [54]:
import matplotlib.pyplot as plt

plt.plot(hist.history["loss"])
plt.plot(hist.history["acc"])
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["loss", "acc"], loc="upper right")
plt.show()

In [55]:

#Probability that first movie in final match up will win:
model.predict(X_final) 


array([[0.9124464]], dtype=float32)

# Logistic Regression
A much simpler approach a little more robust against sample size

In [56]:
from sklearn import linear_model

In [57]:
y_train = y_train.as_matrix()
y_train = np.squeeze(y_train)
y_train.shape

(57,)

In [58]:
model = linear_model.LogisticRegression()
model = model.fit(X_train, y_train)

In [59]:
# check the accuracy on the test set -- seems to be a bit worse than NN
model.score(X_test, y_test)

0.6829268292682927

In [60]:
#confusion matrix

print(metrics.confusion_matrix(y_test, model.predict(X_test)>0.5)) 

[[13  6]
 [ 7 15]]


In [61]:
X_final

,return_1,popularity_1,vote_count_1,vote_average_1,return_2,popularity_2,vote_count_2,vote_average_2
0,0.000002,0.258803,0.616013,0.845455,7.791670e-07,0.024822,0.14784,0.790909


In [62]:
#probability of 0, and probability of 1
#in other words, probability fargo wins, probability pulp fiction wins
model.predict_proba(X_final) 

array([[0.33984156, 0.66015844]])

In [63]:
model.predict(X_final) 

array([1])